## Feature Extraction with Silence Removal

In [3]:
from pydub import AudioSegment, silence
import opensmile
import os
import pandas as pd
from tqdm import tqdm

audio_dir = "wav_files"  # 각 Participant_ID.wav 파일이 있는 폴더
output_dir = "trimmed_audio/"
os.makedirs(output_dir, exist_ok=True)

# Silence 제거 및 저장
def remove_silence(input_path, output_path, silence_thresh=-40, min_silence_len=500):
    audio = AudioSegment.from_wav(input_path)
    chunks = silence.split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    if not chunks:
        print(f"[warning] no voiced chunks in {input_path}, copying as is.")
        audio.export(output_path, format="wav")
        return
    voiced = sum(chunks)
    voiced.export(output_path, format="wav")

In [6]:
info_path = 'data/info_df.csv'
info_df = pd.read_csv(info_path)

for pid in tqdm(info_df["Participant_ID"].unique()):
    input_path = os.path.join(audio_dir, f"{pid}_AUDIO.wav")
    output_path = os.path.join(output_dir, f"{pid}_AUDIO.wav")
    if os.path.exists(input_path):
        remove_silence(input_path, output_path)

 13%|█▎        | 24/189 [03:18<21:45,  7.91s/it]

[warning] no voiced chunks in wav_files\323_AUDIO.wav, copying as is.


 29%|██▊       | 54/189 [07:18<15:32,  6.91s/it]

[warning] no voiced chunks in wav_files\354_AUDIO.wav, copying as is.


 71%|███████   | 134/189 [19:31<07:55,  8.64s/it]

[warning] no voiced chunks in wav_files\436_AUDIO.wav, copying as is.


100%|██████████| 189/189 [27:58<00:00,  8.88s/it]


In [7]:
def extract_opensmile_features(df: pd.DataFrame) -> pd.DataFrame:
    temp_df = df.copy()
    temp_df['wav_path'] = 'trimmed_audio/' + info_df['Participant_ID'].astype(str) + '_AUDIO.wav'
    smile = opensmile.Smile(
                feature_set = opensmile.FeatureSet.eGeMAPSv02, # eGeMAPSv02 does not use LLD, other options: ComParE_2016, ...
                feature_level = opensmile.FeatureLevel.Functionals)

    files = temp_df.wav_path
    temp_features = smile.process_files(files)
    temp_features.reset_index(inplace=True)
    temp_features.drop(columns=['file', 'start', 'end'], axis=1, inplace=True)
    temp_features.insert(0, 'Participant_ID', df['Participant_ID'].values)
    return temp_features

In [8]:
smile_path = "data/smile_features_silence_removed.csv"

if not os.path.exists(smile_path):
    print(f'{smile_path} not found, extracting OpenSMILE features...')
    smile_df = extract_opensmile_features(info_df)
    smile_df.to_csv(smile_path, index=False)
else:
    print(f'{smile_path} found, loading...')
    smile_df = pd.read_csv(smile_path)

data/smile_features_silence_removed.csv not found, extracting OpenSMILE features...
